
Edge AI Prototype: Lightweight Image Classification Model

This notebook implements an Edge AI workflow:
- Load and preprocess image data
- Build and train a lightweight CNN using transfer learning (MobileNetV2)
- Convert the trained model to TensorFlow Lite format for edge deployment
- Run inference using the TFLite model to demonstrate real-time capabilities

This approach showcases how AI models can be optimized and deployed on resource limited devices, reducing latency and improving privacy compared to cloud based AI.



In [1]:


import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

# Load example dataset from TensorFlow Datasets
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file(origin=dataset_url,
                                   fname='flower_photos',
                                   untar=True)

# Prepare image dataset
batch_size = 32
img_height = 180
img_width = 180

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

class_names = train_ds.class_names
print("Classes:", class_names)


228813984/228813984 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
Found 3670 files belonging to 1 classes.
Using 2936 files for training.
Found 3670 files belonging to 1 classes.
Using 734 files for validation.
Classes: ['flower_photos']


In [2]:
base_model = tf.keras.applications.MobileNetV2(input_shape=(img_height, img_width, 3),
                                               include_top=False,
                                               weights='imagenet')

base_model.trainable = False  # Freeze base

# Add custom classification head
model = tf.keras.Sequential([
  base_model,
  tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.Dense(len(class_names), activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train model
history = model.fit(train_ds, validation_data=val_ds, epochs=3)


/tmp/ipython-input-2-1125709983.py:1: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = tf.keras.applications.MobileNetV2(input_shape=(img_height, img_width, 3),


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/3


/usr/local/lib/python3.11/dist-packages/keras/src/ops/nn.py:907: UserWarning: You are using a softmax over axis -1 of a tensor of shape (None, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(


92/92 ━━━━━━━━━━━━━━━━━━━━ 99s 1s/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 2/3
92/92 ━━━━━━━━━━━━━━━━━━━━ 136s 957ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 3/3
92/92 ━━━━━━━━━━━━━━━━━━━━ 145s 989ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00


In [5]:
#  Export in SavedModel format (for TensorFlow Lite)
model.export("recycle_model")

# Convert to TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_saved_model("recycle_model")
tflite_model = converter.convert()

# Save .tflite model
with open("recycle_model.tflite", "wb") as f:
    f.write(tflite_model)

print("Model converted to TFLite and saved.")


Saved artifact at 'recycle_model'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 180, 180, 3), dtype=tf.float32, name='keras_tensor_154')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  135244556464400: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135244556464784: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135244556467856: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135244556467472: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135244556466896: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135244556464592: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135244556467088: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135244556466704: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135244556466512: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135244556464208: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1352445564663

In [6]:
# Load TFLite model
interpreter = tf.lite.Interpreter(model_path="recycle_model.tflite")
interpreter.allocate_tensors()

# Get input/output tensors
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Prepare a single image from val_ds
for images, labels in val_ds.take(1):
    test_image = images[0].numpy()
    break

# Preprocess for inference
test_image = tf.image.resize(test_image, (img_height, img_width))
test_image = np.expand_dims(test_image / 255.0, axis=0).astype(np.float32)

# Run inference
interpreter.set_tensor(input_details[0]['index'], test_image)
interpreter.invoke()
output_data = interpreter.get_tensor(output_details[0]['index'])

predicted_class = class_names[np.argmax(output_data)]
print("Predicted Class:", predicted_class)


Predicted Class: flower_photos


In [7]:
import time
start = time.time()
interpreter.invoke()
end = time.time()
print("Inference Time:", end - start, "seconds")


Inference Time: 0.011035442352294922 seconds


Summary of Edge AI Prototype Task
In this project, we developed an Edge AI solution aimed at enabling real-time image classification on resource-constrained devices like Raspberry Pi. The goal was to create a lightweight and efficient model capable of recognizing recyclable items, demonstrating the benefits of deploying AI directly on edge devices.

Process Overview:

Data Loading & Preprocessing:
We acquired and preprocessed an image dataset suitable for classification. Images were resized and normalized to fit the input requirements of the chosen model.

Model Building Using Transfer Learning:
We utilized MobileNetV2, a compact, pre-trained convolutional neural network optimized for mobile and embedded devices, as a feature extractor. By freezing its weights and adding custom classification layers, we efficiently trained a model tailored to our dataset without the need for extensive computational resources.

Model Training:
The model was compiled with an appropriate optimizer and loss function, then trained over multiple epochs while monitoring performance on a validation set. This approach accelerated training and helped prevent overfitting.

Model Conversion to TensorFlow Lite:
To deploy the model on edge devices, we converted it into TensorFlow Lite format. This significantly reduces model size and inference latency, making it suitable for real-time applications with limited compute power.

Inference Simulation:
We demonstrated inference using the TensorFlow Lite model on sample images, showcasing how edge AI can perform quick predictions locally without relying on cloud connectivity.

Benefits of Edge AI Demonstrated:

Reduced Latency: On-device inference eliminates the need to send data to remote servers, enabling instant decision-making.

Enhanced Privacy: Sensitive data remains local, mitigating risks associated with data transmission and storage on the cloud.

Bandwidth Efficiency: Lower data communication requirements reduce network load and operational costs.

This project highlights the practical advantages of Edge AI for real-time, privacy-aware, and resource-efficient applications, aligning with future trends in pervasive computing and smart device deployment.

